# data1 : 건물 주소  |  data2 : 공영주차장 

### 1) 군집 중앙값과 data2 데이터별 거리 중 가장 가까운 군집

In [74]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import pairwise_distances

# data1 : 건물 주소  |  data2 : 공영주차장 
# log변환 - 로버스트 스케일링 - umap (correlation) - HDBSCAN_v3
df = pd.read_csv("C:/Python_practice/DL_project/gwangjin_contest/EDA_and_Model/4.Clustering_Tuning/third_priority/LogAndRobustScaled_correlationUMAP_HDBSCAN_v3.csv", index_col=0)

# data1에 대한 HDBSCAN 군집화 결과 저장
data1 = df[['HDBSCAN_cluster_without_noise']]
data1

,HDBSCAN_cluster_without_noise
지번주소,
서울특별시 광진구 광장동 104,3
서울특별시 광진구 광장동 105,3
서울특별시 광진구 광장동 106,3
서울특별시 광진구 광장동 107,3
서울특별시 광진구 광장동 108,3
...,...
서울특별시 광진구 화양동 94-61,5
서울특별시 광진구 화양동 94-73,5
서울특별시 광진구 화양동 95-3,4


### data1 의 칼럼 다시 추가

In [75]:
data1_col_df = pd.read_csv("C:/Python_practice/DL_project/gwangjin_contest/EDA_and_Model/train_df_scaling.csv", index_col=0)
data1_col_df.head(5)

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,,
서울특별시 광진구 광장동 102,7067.7,136,2,51,2211.8,6324000.0,1144,41
서울특별시 광진구 광장동 104,7067.7,136,2,21,172.1,5097000.0,1166,41
서울특별시 광진구 광장동 105,7067.7,136,2,3,848.8,5202000.0,1068,41
서울특별시 광진구 광장동 106,7067.7,136,2,27,452.8,5097000.0,1038,41
서울특별시 광진구 광장동 107,7067.7,136,2,50,403.9,5793000.0,1011,42


In [76]:
data1_merged = pd.merge(data1, data1_col_df, on='지번주소', how='left')
data1_merged.head(5)

,HDBSCAN_cluster_without_noise,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,주차장과의최단거리,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,,,
서울특별시 광진구 광장동 104,3,7067.7,136,2,21,172.1,5097000.0,1166,41
서울특별시 광진구 광장동 105,3,7067.7,136,2,3,848.8,5202000.0,1068,41
서울특별시 광진구 광장동 106,3,7067.7,136,2,27,452.8,5097000.0,1038,41
서울특별시 광진구 광장동 107,3,7067.7,136,2,50,403.9,5793000.0,1011,42
서울특별시 광진구 광장동 108,3,1458.4,340,2,30,567.7,7285000.0,1142,42


In [77]:
data1_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15652 entries, 서울특별시 광진구 광장동  104 to 서울특별시 광진구 화양동  99
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   HDBSCAN_cluster_without_noise  15652 non-null  int64  
 1   인구밀도                           15652 non-null  float64
 2   총 사업체수                         15652 non-null  int64  
 3   반경 1km 이내 불법주정차 단속 수           15652 non-null  int64  
 4   주차장과의최단거리                      15652 non-null  int64  
 5   토지면적                           15652 non-null  float64
 6   공시지가(원/면적)                     15652 non-null  float64
 7   역과의최단거리                        15652 non-null  int64  
 8   반경 1km 이내 주차장수                 15652 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 1.2+ MB


In [78]:
# 칼럼 수가 다름
del data1_merged['주차장과의최단거리']

In [79]:
# 군집별 중앙값 계산
cluster_medians = data1_merged.groupby('HDBSCAN_cluster_without_noise').median()
cluster_medians

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
HDBSCAN_cluster_without_noise,,,,,,,
0,43347.3,28.0,30.0,148.80,4200000.0,634.0,131.0
1,43580.2,35.0,12.0,164.05,4410000.0,468.0,87.0
2,47909.9,27.0,12.0,155.40,4698000.0,1135.0,112.0
3,30470.0,70.0,16.0,158.90,5339000.0,978.0,111.0
4,46981.0,31.0,26.0,151.10,4890000.0,680.0,65.0
5,22499.4,91.0,24.0,171.60,5536000.0,548.0,86.0


### data2 로그 변환 + 로버스트 스케일링

In [80]:
# data2 불러오기
data2_col = ['지번주소', '인구밀도', '총 사업체수', '반경 1km 이내 불법주정차 단속 수', '토지면적', '공시지가(원/면적)', '역과의최단거리', '반경 1km 이내 주차장수']
data2 = pd.read_excel('C:/Python_practice/DL_project/gwangjin_contest/광진구_공영주차장_230424.xlsx', index_col=False)[data2_col]
data2.set_index('지번주소', inplace=True)
data2.head(5)

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,
서울특별시 광진구 광장동 105,7067.7,136,1,848.8,5202000,334,40
서울특별시 광진구 광장동 110,1458.4,340,1,672.7,6324000,1141,41
서울특별시 광진구 광장동 112,1458.4,340,1,1587.4,7102000,1122,41
서울특별시 광진구 광장동 126,1458.4,340,1,512.0,6037000,1028,41
서울특별시 광진구 광장동 127,1458.4,340,2,856.8,7720000,1000,41


In [81]:
# 중복된 인덱스를 가진 행 찾기
duplicated_index_rows = data2[data2.index.duplicated()]

# 중복된 인덱스를 가진 행 출력
print(duplicated_index_rows)

                        인구밀도  총 사업체수  반경 1km 이내 불법주정차 단속 수   토지면적  공시지가(원/면적)  \
지번주소                                                                            
서울특별시 광진구 구의동 237-6  10790.3      36                    18  906.1     5640000   

                     역과의최단거리  반경 1km 이내 주차장수  
지번주소                                          
서울특별시 광진구 구의동 237-6      686              87  


In [82]:
# 중복 인덱스를 제거하고 새 데이터 프레임 생성
data2_no_duplicates = data2.loc[~data2.index.duplicated(keep='first')]
data2_no_duplicates

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,
서울특별시 광진구 광장동 105,7067.7,136,1,848.8,5202000,334,40
서울특별시 광진구 광장동 110,1458.4,340,1,672.7,6324000,1141,41
서울특별시 광진구 광장동 112,1458.4,340,1,1587.4,7102000,1122,41
서울특별시 광진구 광장동 126,1458.4,340,1,512.0,6037000,1028,41
서울특별시 광진구 광장동 127,1458.4,340,2,856.8,7720000,1000,41
...,...,...,...,...,...,...,...
서울특별시 광진구 화양동 6-2,18152.8,311,11,111.6,33834000,232,111
서울특별시 광진구 화양동 63-2,58911.9,12,13,664.0,3717000,606,71
서울특별시 광진구 화양동 7-3,18152.8,311,10,2291.3,18760000,79,107


In [83]:
# data2에 대해 로그 변환 적용
def log_transform(dataframe, columns):
    for column in columns:
        filtered = dataframe[column][dataframe[column] > 0] # filter out negative or zero values
        dataframe.loc[filtered.index, column] = np.log(filtered) # apply log transformation
    return dataframe

log_df = log_transform(data2_no_duplicates, data2_no_duplicates.columns)
log_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 474 entries, 서울특별시 광진구 광장동 105 to 서울특별시 광진구 화양동 8-36
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   인구밀도                  474 non-null    float64
 1   총 사업체수                474 non-null    float64
 2   반경 1km 이내 불법주정차 단속 수  474 non-null    float64
 3   토지면적                  474 non-null    float64
 4   공시지가(원/면적)            474 non-null    float64
 5   역과의최단거리               474 non-null    float64
 6   반경 1km 이내 주차장수        474 non-null    float64
dtypes: float64(7)
memory usage: 45.8+ KB


C:\Users\dohee\AppData\Local\Temp\ipykernel_6752\1245759367.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.loc[filtered.index, column] = np.log(filtered) # apply log transformation


In [84]:
# 로버스트 적용
from sklearn.preprocessing import RobustScaler

# 로버스트 스케일러 객체 생성
robust_scaler = RobustScaler()

# 로버스트 스케일링 적용
log_and_robust_scaled_data2 = robust_scaler.fit_transform(log_df)

# 결과를 DataFrame으로 변환
log_and_robust_scaled_data2 = pd.DataFrame(log_and_robust_scaled_data2, columns=log_df.columns, index = log_df.index)
log_and_robust_scaled_data2.head(5)

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,
서울특별시 광진구 광장동 105,-1.568483,0.818332,-2.299527,0.669026,0.089785,-1.011443,-1.685186
서울특별시 광진구 광장동 110,-3.239565,1.552858,-2.299527,0.556962,0.445100,0.729132,-1.635227
서울특별시 광진구 광장동 112,-3.239565,1.552858,-2.299527,0.970737,0.656177,0.705341,-1.635227
서울특별시 광진구 광장동 126,-3.239565,1.552858,-2.299527,0.425404,0.360605,0.581374,-1.635227
서울특별시 광진구 광장동 127,-3.239565,1.552858,-1.574107,0.673547,0.807971,0.542248,-1.635227


In [110]:
distances.shape

(474, 6)

In [111]:
closest_clusters.shape

(474,)

In [85]:
# 각 data2 포인트와 군집 중앙값 사이의 거리 계산
distances = pairwise_distances(log_and_robust_scaled_data2, cluster_medians)

# 가장 가까운 군집 찾기
closest_clusters = np.argmin(distances, axis=1)
closest_clusters 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 군집 0이 가장 유사한 군집 => 데이터 저장

In [96]:
cluster0_data1 = data1_merged[data1_merged['HDBSCAN_cluster_without_noise'] == 0]
cluster0_data1

,HDBSCAN_cluster_without_noise,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
지번주소,,,,,,,,
서울특별시 광진구 구의동 57-2,0,62794.7,8,12,144.5,4905000.0,555,88
서울특별시 광진구 구의동 57-41,0,62794.7,8,12,158.3,4854000.0,526,87
서울특별시 광진구 구의동 612-11,0,21451.4,17,20,136.0,3392000.0,709,67
서울특별시 광진구 구의동 612-12,0,21451.4,17,20,46.0,3392000.0,721,66
서울특별시 광진구 구의동 612-13,0,21451.4,17,20,95.0,3357000.0,625,66
...,...,...,...,...,...,...,...,...
서울특별시 광진구 화양동 427-4,0,32646.5,56,32,202.0,4018000.0,739,83
서울특별시 광진구 화양동 427-5,0,32646.5,56,30,106.0,4018000.0,703,82
서울특별시 광진구 화양동 430-4,0,28136.4,39,34,225.0,5109000.0,699,87


In [97]:
# 위경도 추가
point_col = ['지번주소', '위도', '경도']
place_point_df = pd.read_excel("C:\Python_practice\DL_project\gwangjin_contest\광진구_데이터프레임_230425.xlsx")[point_col]
place_point_df.set_index('지번주소', inplace=True)

cluster0_data1 = pd.merge(cluster0_data1, place_point_df, on='지번주소', how='left')
cluster0_data1.head(3)

,HDBSCAN_cluster_without_noise,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수,위도,경도
지번주소,,,,,,,,,,
서울특별시 광진구 구의동 57-2,0,62794.7,8,12,144.5,4905000.0,555,88,37.550094,127.093240
서울특별시 광진구 구의동 57-41,0,62794.7,8,12,158.3,4854000.0,526,87,37.550133,127.093117
서울특별시 광진구 구의동 612-11,0,21451.4,17,20,136.0,3392000.0,709,67,37.548504,127.085626


In [98]:
# 군집0인 지번 주소 저장
cluster0_addresses = cluster0_data1.index

<hr>

### 2) 전체 평균과 군집 평균이 비슷한 군집
군집 평균값과 data2 데이터 전체 평균의 차이가 가장 작은 군집

In [113]:
# data1에 대한 군집별 평균 계산
cluster_mean = data1_merged.groupby('HDBSCAN_cluster_without_noise').mean()

# data2 전체 평균 계산
data2_mean = log_and_robust_scaled_data2.copy()
data2_overall_mean = data2_mean.mean()

# 군집별 평균과 전체 평균 간 차이 계산
mean_differences = cluster_mean.subtract(data2_overall_mean)

# 차이의 절대값을 취한 후 각 군집별로 합산
sum_abs_mean_differences = mean_differences.abs().sum(axis=1)

# 차이가 가장 작은 군집 찾기
similar_cluster = sum_abs_mean_differences.idxmin()
similar_cluster

0

In [114]:
cluster_mean

,인구밀도,총 사업체수,반경 1km 이내 불법주정차 단속 수,토지면적,공시지가(원/면적),역과의최단거리,반경 1km 이내 주차장수
HDBSCAN_cluster_without_noise,,,,,,,
0,42992.808357,32.616053,28.619363,168.876458,4.251504e+06,635.762929,128.827886
1,42871.070011,42.346529,12.425163,191.694089,4.347755e+06,461.606833,89.242408
2,46360.590701,29.073409,13.769440,173.666449,4.484971e+06,1099.684067,111.196302
3,28565.361562,87.418499,15.553450,187.785595,5.388153e+06,977.915845,101.738438
4,47714.765181,32.082329,25.643775,166.267309,4.915922e+06,678.877912,62.621285
5,23731.506303,116.863138,23.498694,652.769145,6.235170e+06,568.083058,89.463869


In [115]:
sum_abs_mean_differences

HDBSCAN_cluster_without_noise
0    4.295493e+06
1    4.391424e+06
2    4.532760e+06
3    5.418090e+06
4    4.964603e+06
5    6.260353e+06
dtype: float64

<hr>

### folium 라이브러리 사용해서 map에 인구밀도가 높은 top 100 찍어보기

In [102]:
#pip install folium

In [108]:
import folium

# 인구 밀도가 높은 순으로 정렬
sorted_cluster0_data1 = cluster0_data1.sort_values(by='인구밀도', ascending=False) # 내림차순

# 최대 100개 선택
top100_cluster0_data1 = sorted_cluster0_data1.head(100)

# 위도와 경도의 평균값을 사용하여 지도의 초기 위치 설정
map_center_lat = top100_cluster0_data1['위도'].mean()
map_center_lon = top100_cluster0_data1['경도'].mean()

# 지도 생성
m = folium.Map(location=[map_center_lat, map_center_lon], zoom_start=14)

# 지도에 마커 추가
for index, row in top100_cluster0_data1.iterrows():
    lat, lon = row['위도'], row['경도']
    folium.Marker(location=[lat, lon]).add_to(m)

# 지도 출력
m